# Persona Intent Parsing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from persona.preprocess import prepare_json_data, pad_sequences, integer_encode, one_hot_encode, one_hot_encode_target
import random

## Preprocess the data

In [3]:
MAX_SEQ_LEN = 15
input_word_model, output_word_model, pairs = \
    prepare_json_data('input', 'output', './data/persona.intents.json', MAX_SEQ_LEN)

READ 816 sentence pairs
Trimmed to 816 sentence pairs
Counting words...
Counted Words:
input 244
output 53


In [4]:
input_seqs = [pair[0] for pair in pairs]
intent = [pair[1] for pair in pairs]
for i in range(10):
    rando = random.randint(0, len(pairs))
    print(input_seqs[rando],"-->" ,intent[rando])

application using top ram --> mem.max.process
stop the alarm --> alarm.stop
remove alarm --> alarm.remove
how do i start {skill} --> how.do.i.activate
what alarms do i have --> alarm.list
list the things to do --> show
disable the alarm --> alarm.remove
begin audio recording --> record.begin
what app is using all the mem --> mem.max.process
what s going on --> whats.up


#### Pad the input
This process will add the PAD, SOS, EOS, and UNK tokens to sequence

In [5]:
padded_input = pad_sequences(input_seqs, MAX_SEQ_LEN)
print(padded_input[1])

['SOS', 'why', 'did', '{skill}', 'fail', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']


#### One hot encode

In [6]:
one_hot_input = one_hot_encode(padded_input, input_word_model, MAX_SEQ_LEN)
one_hot_input.shape

(816, 15, 244)

In [7]:
one_hot_output = one_hot_encode([intent], output_word_model, len(intent))[0]
print(one_hot_output.shape)

(816, 53)


## Build and Train the Model

In [8]:
from persona.model.intent import IntentModel

Using TensorFlow backend.


In [9]:
model = IntentModel(one_hot_input, one_hot_output, input_word_model.n_words, output_word_model.n_words)
model.train(summary=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               190976    
_________________________________________________________________
dense_1 (Dense)              (None, 53)                6837      
Total params: 197,813
Trainable params: 197,813
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
816/816 [==============================] - 1s - loss: 3.8423     
Epoch 2/100
816/816 [==============================] - 0s - loss: 3.2222     
Epoch 3/100
816/816 [==============================] - 0s - loss: 3.1586     
Epoch 4/100
816/816 [==============================] - 0s - loss: 3.0804     
Epoch 5/100
816/816 [==============================] - 0s - loss: 2.9361     
Epoch 6/100
816/816 [==============================] - 0s - loss: 2.9654     
Epoch 7/100
816/816 [==============================] - 0s - loss

816/816 [==============================] - 0s - loss: 0.5674     
Epoch 98/100
816/816 [==============================] - 0s - loss: 0.0815     
Epoch 99/100
816/816 [==============================] - 0s - loss: 0.3837     
Epoch 100/100
816/816 [==============================] - 0s - loss: 0.2079     


In [10]:
from persona.preprocess import pad_sequence

try:
    while True:
        _input = input("input: ")
        padded_input = [pad_sequence(_input, MAX_SEQ_LEN).split()]
        one_hot = one_hot_encode(padded_input, input_word_model, MAX_SEQ_LEN)
        prediction, confidence = model.decode(one_hot, output_word_model)
        print("intent: ", prediction, confidence)
except KeyboardInterrupt:
    pass

input:  stop the alarm
intent:  alarm.stop 0.608037
input: remove alarm
intent:  alarm.remove 0.585099
input: what is the weather
intent:  weather.current 0.965198
input: what time is it
intent:  time 0.978605
input: cpu usage
intent:  cpu.total.usage 0.781115
input: what app is using mem
intent:  mem.max.process 1.0
